# ชุด Source code นี้ใช้ทดสอบ ผลลัพธ์ของ "python ingre.py"
# หรือการรัน script phython เพื่ออ่านข้อมูลจากไฟล์เอกสาร และ เก็บลงใน ChromaDb (AI DB)

In [1]:
from constants import (
    get_default_ai_db_collection_name,
    get_default_embedding,
    load_chroma_database,
    is_exists_collection_name,
    AI_DB_SEARCH_RESULT_RECORD,
    AI_DB_METADATA_DOCUMENT_SOURCE_NAME,
    AI_DB_METADATA_INTERNAL_IDX_NAME
)

db = load_chroma_database()
collection = db.get_collection(name=get_default_ai_db_collection_name(), embedding_function=get_default_embedding())

d:\Program Files\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import chromadb

def query_from_funetune(query_str: str, db: chromadb.API) -> str:
    """
    ค้นหาเนื้อหาภายในเอกสาร ที่เก็บภายใน AI DB มีรูปประโยคใดบ้าง ที่ใกล้เคียงกับ query_str
    """
    ret_str = ''
    if is_exists_collection_name(collection_name=get_default_ai_db_collection_name(), db=db) == False:
        return ret_str
    
    collection = db.get_collection(
        name=get_default_ai_db_collection_name(), 
        embedding_function=get_default_embedding()
    )

    # Trim space ซ้าย/ขวา ก่อนใช้เป็นเงื่อนไขในการค้นหา เพราะทุกตัวอักษรถูกนำไปคิดเป็น เงื่อนไขทั้งหมด
    query_str = query_str.strip()

    # ค้นหารูปแบบประโยคที่เฉพาะเจาะจง
    parent_out = collection.query(
        query_texts=[query_str],
        n_results=AI_DB_SEARCH_RESULT_RECORD,
        where_document={
            '$contains': query_str
        }
    )
    print(parent_out)
    print(parent_out['metadatas'])

    if len(parent_out['documents'][0]) == 0:
        # ถ้าค้นหาด้วยเงื่อนไข รูปแบบประโยคที่เฉพาะเจาะจงไม่เจอ ให้ค้นหาแบบ distancing
        # คือ ให้ AI DB จัดลำดับข้อความที่มีความน่าจะเป็น ที่ใกล้เคียงกับ ประโยคคำถามมากที่สุด ไปหาน้อย (Distancing)
        out = collection.query(
            query_texts=[query_str],
            n_results=5
        )
        
        print(out)
        ret_str = '\n'.join([document_content for document_content in out['documents'][0]]) if len(out['documents'][0]) > 0 else ''
        return ret_str
    

    ret_str = ''.join(text for text in parent_out['documents'][0])

    # ค้นหา Chunk ในลำดับถัดไปจาก parent_out
    # เนื่องจาก Chunk เป็นการตัดชุดของข้อความออกเป็นส่วนๆ มีโอกาสที่ คำตอบ ของคำถามจะไปอยู่ใช้ Chunk ลำดับถัดไป
    doc_source = parent_out['metadatas'][0][0][AI_DB_METADATA_DOCUMENT_SOURCE_NAME]
    doc_internal_idx = parent_out['metadatas'][0][0][AI_DB_METADATA_INTERNAL_IDX_NAME]
    doc_internal_idx1 = f"idx_{int(doc_internal_idx.replace('idx_', '')) + 1}"
    

    out = collection.query(
        query_texts=[query_str],
        n_results=1,
        where={
            '$and':[
                {
                    AI_DB_METADATA_DOCUMENT_SOURCE_NAME: {
                        '$eq': doc_source
                    }
                },
                {
                    AI_DB_METADATA_INTERNAL_IDX_NAME: {
                        '$eq': doc_internal_idx1
                    }
                }
            ]
        }
    )

    print(f'doc_source: {doc_source}, doc_internal_idx: {doc_internal_idx}, doc_internal_idx1: {doc_internal_idx1}')
    print(out)
    ret_str = f"{ret_str}\n{out['documents'][0][0]}" if len(out['documents'][0]) > 0 else ret_str
    return ret_str

In [3]:
#print("'".encode('utf-8'))
query_str = 'Please describe information about deposit type "Cash"'
#print(AI_DB_SEARCH_RESULT_RECORD)
#print(query_str)
ret = query_from_funetune(query_str=query_str, db=db)
print(ret)



{'ids': [[]], 'embeddings': None, 'documents': [[]], 'metadatas': [[]], 'distances': [[]]}
[[]]
{'ids': [['source_documents-single\\Cash Balance Deposit_vn.pdf-idx_18', 'source_documents-single\\Cash Balance Deposit_vn.pdf-idx_10']], 'embeddings': None, 'documents': [['If customer deposits both cash and cheque each deposit types will be deposited separately due to cash balance system has a several types of deposit. For example, the refer type of cash deposit is', 'of deposition, if customers pay by cheque or cash, users has to fill details of deposit slip. If customers pay by cash or another pay type which is qualifies to cash, users don’t have to fill details']], 'metadatas': [[{'internal_idx': 'idx_18', 'source': 'source_documents-single\\Cash Balance Deposit_vn.pdf'}, {'internal_idx': 'idx_10', 'source': 'source_documents-single\\Cash Balance Deposit_vn.pdf'}]], 'distances': [[0.3064601421356201, 0.33947741985321045]]}
If customer deposits both cash and cheque each deposit types wil

In [ ]:
query_text = 'What does Lock(sell) mean?'
ret = collection.query(
    query_texts=[query_text],

    # อ้างอิงเอกสารจาก Link: https://docs.trychroma.com/usage-guide#using-where-filters
    where={
        '$or':[
            {
                'internal_idx':{
                    '$eq': 'idx_9'
                },
            },
             {
                'internal_idx':{
                    '$eq': 'idx_8'
                },
            }
        ]
    },
    # where_document={
    #     '$contains': query_text
    # }
)

print(ret)
print(ret['documents'][0])
print(ret['documents'][0][0])
print(ret['documents'][0][1])